In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 29 00:18:39 2018
@filename: Diamonds-EDA-VDA
@dataset: diamonds-m.csv
@author: cyruslentin
"""

hides all warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

imports<br>
pandas 

In [ ]:
import pandas as pd
# numpy
import numpy as np
# matplotlib 
import matplotlib.pyplot as plt
#%matplotlib inline
plt.style.use('ggplot')
# seaborn
import seaborn as sns
# utils
import utils

############################################################<br>
Read Data <br>
############################################################

In [ ]:
df = pd.read_csv('./data_files/diamonds-m.csv')

############################################################<br>
structure<br>
############################################################

In [ ]:
print("\n*** Structure ***")
print(df.info())

############################################################<br>
data type of columns<br>
############################################################

In [ ]:
dsRet = pd.Series() 
vColList = df.columns.to_list()
for vCol in vColList:
    dsRet[vCol] = df[vCol].dtypes
print("\n*** Data Type ***")
print(dsRet)
# store for future use
dsDataType = dsRet

############################################################<br>
Length Of Alpha Numeric Cols<br>
############################################################

In [ ]:
dfTmp = pd.DataFrame()
dsRet = pd.Series() 
vColList = df.columns.to_list()
for vCol in vColList:
    if df[vCol].dtypes != 'object':
        continue
    dfTmp["tmp"] = df[vCol].str.len()
    lColLen = max(dfTmp['tmp'])
    dsRet[vCol] = lColLen
print("\n*** Lenght Of Alpha Cols ***")
print(dsRet)
    


<br>
Precision is the number of digits in a number. <br>
Scale is the number of digits to the right of the decimal point in a number. <br>
For example, the number 123.45 has a precision of 5 and a scale of 2. <br>
<br>
############################################################<br>
Precision Of Numeric Cols<br>
############################################################

In [ ]:
dfTmp = pd.DataFrame()
dsRet = pd.Series() 
vColList = df.columns.to_list()
for vCol in vColList:
    if (df[vCol].dtypes != 'int64' and df[vCol].dtypes != 'float64' ):
        continue
    #print(vCol, df[vCol].dtypes)
    dfTmp[vCol] = df[vCol].astype(str)
    dfTmp[vCol] = dfTmp[vCol].str.len()
    lColLen = max(dfTmp[vCol])
    if (df[vCol].dtypes == 'int64'):
       dsRet[vCol] = lColLen 
    else:
       dsRet[vCol] = lColLen - 1
print("\n*** Precision Of Numeric Cols ***")
print(dsRet)

############################################################<br>
Scale Of Numeric Cols<br>
############################################################

In [ ]:
dfTmp = pd.DataFrame()
dsRet = pd.Series() 
vColList = df.columns.to_list()
for vCol in vColList:
    if (df[vCol].dtypes != 'int64' and df[vCol].dtypes != 'float64' ):
        continue
    if df[vCol].dtypes == 'int64':
         dsRet[vCol] = 0
         continue
    #print(vCol, df[vCol].dtypes)
    dfTmp = df[[vCol]]
    dfTmp = dfTmp.fillna(-1)
    dfTmp[vCol] = dfTmp[vCol] - dfTmp[vCol].astype(int) 
    dfTmp[vCol] = df[vCol].astype(str)
    dfTmp[vCol] = dfTmp[vCol].str.len()
    lColLen = max(dfTmp[vCol])
    dsRet[vCol] = lColLen - 1
print("\n*** Scale Of Numeric Cols ***")
print(dsRet)


<br>
Assumption<br>
Significant Columns are those columns which starts with or ends with <br>
the following terms in ColName<br>
- ID<br>
- Name<br>
- Desc<br>
<br>
############################################################<br>
Significant Columns<br>
############################################################

In [ ]:
lNSigTerms = ['id', 'name', 'desc']
lsRet = []
vColList = df.columns.to_list()
for vCol in vColList:
    bIsNotSig = False
    for vTerm in lNSigTerms:
        if vCol.lower().startswith(vTerm) or vCol.lower().endswith(vTerm): 
            bIsNotSig = True
    if bIsNotSig == False:
        lsRet.append(vCol) 
# store for future use
lSigCols = lsRet       
dfSigCols = df[lSigCols]
print("\n*** Significant Columns ***")
for vCol in lSigCols:
    print(vCol)

############################################################<br>
Null Values<br>
############################################################

In [ ]:
print("\n*** Null Values ***")
print(dfSigCols.isnull().sum())

############################################################<br>
Zero Values<br>
############################################################

In [ ]:
print("\n*** Zero Values ***")
print((dfSigCols==0).sum())

############################################################<br>
Group By Counts Of AlphaNum Cols<br>
to check error<br>
############################################################

In [ ]:
dfRet = pd.DataFrame()
print("\n*** Group By Counts Of Alpha Cols ***")
for vCol in lSigCols:
    if df[vCol].dtypes != 'object':
        continue
    print('\nClass Column:',vCol)
    dfRet = df.groupby(vCol).size()
    dfRet = dfRet.reset_index()
    dfRet.columns = ['Category         ','Count']
    print(dfRet)

############################################################<br>
replace zero with mean<br>
############################################################

In [ ]:
for vCol in lSigCols:
    if (df[vCol].dtypes != 'int64' and df[vCol].dtypes != 'float64' ):
        continue
    vMean = df[vCol].mean()
    df[vCol] = np.where(df[vCol]==0, vMean, df[vCol])
print("\n*** Zero Values With Mean ***")
print(df.mean())
# keep dfSigCols in sync
dfSigCols = df[lSigCols]

############################################################<br>
replace null with mean for numeric cols<br>
############################################################

In [ ]:
df = df.fillna(df.mean())
print("\n*** Null Values With Mean For Numerics ***")
print(df.mean())
# keep dfSigCols in sync
dfSigCols = df[lSigCols]

############################################################<br>
replace null with mode for alpha-numeric cols<br>
############################################################

In [ ]:
print("\n*** Null Values With Mode For AlphaNumerics ***")
for vCol in lSigCols:
#    vCol = 'Species'
    if df[vCol].dtypes != 'object':
        continue
    vMode = df[vCol].mode()
    #if (type(vMode)=='pandas.core.series.Series'):
    if isinstance(vMode, pd.core.series.Series):        
        vMode = vMode[0]
    df[vCol] = np.where(df[vCol].isnull(), vMode, df[vCol])
# keep dfSigCols in sync
dfSigCols = df[lSigCols]
print(df.info())

############################################################<br>
quartile summary<br>
############################################################

In [ ]:
print("\n*** Quartile Summary ***")
print(dfSigCols.describe())

############################################################<br>
range<br>
############################################################

In [ ]:
dsRet = pd.Series() 
for vCol in lSigCols:
    if (df[vCol].dtypes != 'int64' and df[vCol].dtypes != 'float64' ):
        continue
    dsRet[vCol] = df[vCol].max() - df[vCol].min()
print("\n*** Range ***")
print(dsRet)

############################################################<br>
variance<br>
############################################################

In [ ]:
print("\n*** Variance ***")
print(df.var())

############################################################<br>
std dev<br>
############################################################

In [ ]:
print("\n*** Std Dev ***")
print(df.std())

############################################################<br>
check outlier count<br>
############################################################

In [ ]:
print('\n*** Outlier Count ***')
print(utils.OutlierCount(dfSigCols))

############################################################<br>
check outlier index<br>
############################################################

In [ ]:
print('\n*** Outlier Index ***')
print(utils.OutlierIndex(dfSigCols))

############################################################<br>
check outlier values<br>
############################################################

In [ ]:
print('\n*** Outlier Values ***')
print(utils.OutlierValues(dfSigCols))

############################################################<br>
distibution plot<br>
############################################################

In [ ]:
print("\n*** Distribution Plots ***")
for vCol in lSigCols:
    if df[vCol].dtypes != 'object':
        continue
    #print("Col",vCol)
    print("\n*** Column %s " % vCol)
    print(df.groupby(vCol).size())
    plt.figure(figsize=(10,5))
    sns.countplot(df[vCol],label="Count")
    plt.title('Column %s' % vCol)
    plt.show()

############################################################<br>
plot histograms<br>
############################################################

In [ ]:
print("\n*** Histogram Plot ***")
for vCol in lSigCols:
    if (df[vCol].dtypes == 'object'): #and df[vCol].dtypes != 'float64' ):
        continue
    #print("Col",vCol)
    colValues = df[vCol].values
    plt.figure(figsize=(10,5))
    sns.distplot(colValues, bins=7, kde=False, color='b')
    plt.title('Column %s' % vCol)
    plt.ylabel(vCol)
    plt.xlabel('Bins')
    plt.show()

############################################################<br>
plot Boxplot<br>
############################################################

In [ ]:
print("\n*** Box Plot ***")
for vCol in lSigCols:
    if (df[vCol].dtypes == 'object'): #and df[vCol].dtypes != 'float64' ):
        continue
    #print("Col",vCol)
    plt.figure(figsize=(10,5))
    sns.boxplot(y=df[vCol], color='b')
    plt.title('Column %s' % vCol)
    plt.ylabel(vCol)
    plt.xlabel('Bins')
    plt.show()

############################################################<br>
scatterplot<br>
############################################################

In [ ]:
print("\n*** Scatter Plot ***")
vColCount = len(lSigCols)
for i in range(0,vColCount):
    iCol = lSigCols[i]
    if (df[iCol].dtypes == 'object'):  #'int64' and df[iCol].dtypes != 'float64' ):
        continue
    for j in range(i+1,vColCount):
        jCol = lSigCols[j]
        if (df[jCol].dtypes == 'object'): #and df[jCol].dtypes != 'float64' ):
            continue
        #print("\niCol",iCol)
        #print("jCol",jCol)
        plt.figure(figsize=(10,5))
        sns.regplot(data=df, x=iCol, y=jCol, color= 'b', scatter_kws={"s": 5})
        plt.title(iCol + ' v/s ' + jCol)
        plt.show()

############################################################<br>
correlation table<br>
############################################################

In [ ]:
print("\n*** Correlation Table ***")
pd.options.display.float_format = '{:,.2f}'.format
print(dfSigCols.corr())

############################################################<br>
correlation heatmap<br>
############################################################

In [ ]:
print("\n*** Heat Map ***")
plt.figure(figsize=(8,8))
ax = sns.heatmap(dfSigCols.corr(), annot=True, cmap="PiYG")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom+0.5, top-0.5)
plt.show()